## Geometries in Geopandas

Geopandas takes advantage of Shapely's geometric objects. Geometries are stored in a column called *geometry* that is a default column name for
storing geometric information in geopandas.

-  Let's print the first 5 rows of the column 'geometry':

In [30]:
# It is possible to get a specific column by specifying the column name within square brackets []
print(data['geometry'].head())

0    POLYGON ((379394.248 6689991.936, 379389.79 66...
1    POLYGON ((378980.811 6689359.377, 378983.401 6...
2    POLYGON ((378804.766 6689256.471, 378817.107 6...
3    POLYGON ((379229.695 6685025.111, 379233.366 6...
4    POLYGON ((379825.199 6685096.247, 379829.651 6...
Name: geometry, dtype: object


As we can see the `geometry` column contains familiar looking values, namely Shapely `Polygon` -objects that we [learned to use last week](https://automating-gis-processes.github.io/2018/notebooks/L1/geometric-objects.html#Polygon). Since the spatial data is stored as Shapely objects, **it is possible to use all of the functionalities of Shapely module**.

- Let's prove that this really is the case by iterating over a sample of the data, and printing the `area` of first five polygons. 

  - We can iterate over the rows by using the `iterrows()` -function that we learned [during the Lesson 6 of the Geo-Python course](https://geo-python.github.io/2018/notebooks/L6/pandas/advanced-data-processing-with-pandas.html#Iterating-rows-and-using-self-made-functions-in-Pandas).

In [31]:
# Make a selection that contains only the first five rows
selection = data[0:5]

# Iterate over rows and print the area of a Polygon
for index, row in selection.iterrows():
    # Get the area of the polygon
    poly_area = row['geometry'].area
    # Print information for the user
    print("Polygon area at index {index} is: {area:.3f}".format(index=index, area=poly_area))

Polygon area at index 0 is: 76.027
Polygon area at index 1 is: 2652.054
Polygon area at index 2 is: 3185.650
Polygon area at index 3 is: 13075.165
Polygon area at index 4 is: 3980.683


As you might guess from here, all the functionalities of **Pandas**, such as the `iterrows()` function, are directly available in Geopandas without the need to call pandas separately because Geopandas is an **extension** for Pandas. 

- Let's next create a new column into our GeoDataFrame where we calculate and store the areas of individual polygons into that column. Calculating the areas of polygons is really easy in geopandas by using ``GeoDataFrame.area`` attribute. Hence, it is not needed to actually iterate over the rows line by line as we did previously:

In [32]:
# Create a new column called 'area' and assign the area of the Polygons into it
data['area'] = data.area

# Print first 2 rows of the area column
print(data['area'].head(2))

0      76.027392
1    2652.054186
Name: area, dtype: float64


As we can see, the area of our first polygon seems to be approximately `19.396` and `6.146` for the second polygon. They correspond to the ones we saw in previous step when iterating rows, hence, everything seems to work as should.

- Let's check what is the `min`, `max` and `mean` of those areas using familiar functions from our previous Pandas lessions.


In [53]:
# Maximum area
max_area = data['area'].max()

# Minimum area
min_area = data['area'].min()

# Mean area
mean_area = data['area'].mean()

In [54]:
print("Max area: {max} meters".format(max=round(max_area, 0)))
print("Min area: {min} meters".format(min=round(min_area, 0)))
print("Mean area: {mean} meters".format(mean=round(mean_area, 0)))

Max area: 4084558.0 meters
Min area: 1.0 meters
Mean area: 11522.0 meters


## Creating geometries into a GeoDataFrame

Since geopandas takes advantage of Shapely geometric objects, it is possible to create a Shapefile from a scratch by passing Shapely's
geometric objects into the GeoDataFrame. This is useful as it makes it easy to convert e.g. a text file that contains coordinates into a
Shapefile. Next we will see how to create a Shapefile from scratch. 

- Let's create an empty `GeoDataFrame`.

In [34]:
# Import necessary modules first
import geopandas as gpd
from shapely.geometry import Point, Polygon

# Create an empty geopandas GeoDataFrame
newdata = gpd.GeoDataFrame()

# Let's see what we have at the moment
print(newdata)

Empty GeoDataFrame
Columns: []
Index: []


As we can see, the GeoDataFrame is empty since we haven't yet stored any data into it.

- Let's create a new column called `geometry` that will contain our Shapely objects:

In [35]:
# Create a new column called 'geometry' to the GeoDataFrame
newdata['geometry'] = None

# Let's again see what's inside
print(newdata)

Empty GeoDataFrame
Columns: [geometry]
Index: []


Now we have a `geometry` column in our GeoDataFrame but we don't have any data stored yet.

- Let's create a Shapely `Polygon` repsenting the Helsinki Senate square that we can later insert to our GeoDataFrame:

In [36]:
# Coordinates of the Helsinki Senate square in Decimal Degrees
coordinates = [(24.950899, 60.169158), (24.953492, 60.169158), (24.953510, 60.170104), (24.950958, 60.169990)]

# Create a Shapely polygon from the coordinate-tuple list
poly = Polygon(coordinates)

# Let's see what we have
print(poly)

POLYGON ((24.950899 60.169158, 24.953492 60.169158, 24.95351 60.170104, 24.950958 60.16999, 24.950899 60.169158))


Okay, now we have an appropriate `Polygon` -object.

- Let's insert the polygon into our 'geometry' column of our GeoDataFrame at position 0:

In [37]:
# Insert the polygon into 'geometry' -column at index 0
newdata.loc[0, 'geometry'] = poly

# Let's see what we have now
print(newdata)

                                            geometry
0  POLYGON ((24.950899 60.169158, 24.953492 60.16...


Great, now we have a GeoDataFrame with a Polygon that we could already now export to a Shapefile. However, typically you might want to include some useful information with your geometry. 

- Hence, let's add another column to our GeoDataFrame called `location` with text `Senaatintori` that describes the location of the feature.

In [38]:
# Add a new column and insert data 
newdata.loc[0, 'location'] = 'Senaatintori'

# Let's check the data
print(newdata)

                                            geometry      location
0  POLYGON ((24.950899 60.169158, 24.953492 60.16...  Senaatintori


Okay, now we have additional information that is useful for recognicing what the feature represents. 

Before exporting the data it is always good (basically necessary) to **determine the coordinate reference system (projection) for the GeoDataFrame.** GeoDataFrame has an attribute called `.crs` that shows the coordinate system of the data which is empty (None) in our case since we are creating the data from the scratch (more about projection on next tutorial):

In [39]:
print(newdata.crs)

None


-  Let's add a crs for our GeoDataFrame. A Python module called **fiona** has a nice function called ``from_epsg()`` for passing the coordinate reference system information for the GeoDataFrame. Next we will use that and determine the projection to WGS84 (epsg code: 4326):

In [40]:
# Import specific function 'from_epsg' from fiona module
from fiona.crs import from_epsg

# Set the GeoDataFrame's coordinate system to WGS84 (i.e. epsg code 4326)
newdata.crs = from_epsg(4326)

# Let's see how the crs definition looks like
print(newdata.crs)

{'no_defs': True, 'init': 'epsg:4326'}


As we can see, now we have associated coordinate reference system information (i.e. `CRS`) into our `GeoDataFrame`. The CRS information here, is a Python `dictionary` containing necessary values for geopandas to create a `.prj` file for our Shapefile that contains the CRS info. 

-  Finally, we can export the GeoDataFrame using `.to_file()` -function. The function works quite similarly as the export functions in numpy or pandas, but here we only need to provide the output path for the Shapefile. Easy isn't it!:

In [41]:
# Determine the output path for the Shapefile
outfp = "L2_data/Senaatintori.shp"

# Write the data into that Shapefile
newdata.to_file(outfp)

Now we have successfully created a Shapefile from the scratch using only Python programming. Similar approach can be used to for example to read
coordinates from a text file (e.g. points) and create Shapefiles from those automatically.

**TASK:** Check the output Shapefile by reading it with geopandas and make sure that the attribute table and geometry seems correct.